'''
This program selects future-oriented text and cleans text
#Date: Dec 3 2019
#Update: Mar 25, 2019
#update: June 27, 2020
#Author: Carly Knight
'''

### Import packages

In [118]:
import os
from zipfile import ZipFile
import xml.etree.ElementTree as ET
from xml import etree
import csv
import re
import dateutil.parser as parser
from enchant.tokenize import get_tokenizer
import enchant
import pandas as pd
from itertools import chain
import random

## get liwc list


In [119]:
liwclist= "/Users/carlyknight/Dropbox/PROJECTS/ConceptionsofRisk/data/LIWC/liwc2015dict.csv"
lw= pd.read_csv(liwclist)

#select FocusFuture
future= lw.loc[lw['var'] == 'FocusFuture']
futurelist = future['word'].tolist()

#stems versus words
futurelist_stem = [s.strip('*') for s in futurelist if "*" in s]
futurelist_word = [s for s in futurelist if "*" not in s]

#add/remove
futurelist_word.extend(["Future", "will", "is heading", "are heading", "aim","could", "doubt", "has begun", "intend", "intends", "intending", "is expected", "is likely to", "it is hoped", "optimism", "pessimism", "uncertainty", "projected", "certainty", "as yet", "not yet", "may", "outlook", "should", "risk", "risky", "risk-taking", "is planned", "is planning", "is scheduled", "a planned", "the planned", "optimistic", "pessimistic", "next generation", "well positioned", "next-generation", "well-positioned", "is scheduled", "are scheduled"])
futurelist_word.remove("plan")
futurelist_word.remove("plans")
futurelist_word.remove("planner")
futurelist_word.remove("planning")

futurelist_stem.extend(["confident"])
futurelist_stem.remove("headin")
futurelist_stem.remove("prepar")

#notwordlist
notwordlist=["shareholders will", "Shareholders will", "will be of interest", "good will", "GOOD WILL", "Good will", "you will", "was expected", "were expected", "TABLE OF CONTENTS", "table of contents", "should have", "could have", "would have", "should not have", "could not have", "would not have", "may have", "accounting standards"]

#financial statement list
financialstatement = ["Notes to Consolidated Financial Statements", "Notes to Financial Statements"]

#paragrapsh to exclude list
phrasetoexclude = financialstatement + ['financial accounting', "the following table", "transfer agent", "annual meeting of shareholders", "annual meeting of stockholders", "next annual meeting"]

#test examples
example_list = ["possible"]
example_container_lists = [ [] for i in range(len(example_list)) ]

print(futurelist_word)
print(futurelist_stem)
print(notwordlist)
#Other potential options:
#Risk, Money, Quant, Verb, Tentat, Certain, Time

['ahead', 'anticipation', 'approaching', 'attainable', 'coming', 'eventual', 'eventually', 'fate', 'fated', 'fates', 'feasible', 'finna', 'fixin', "fixin'", 'foresight', 'forseeable', 'forthcoming', 'going', 'gon', 'gonna', 'gotta', 'gunna', "he'll", 'henceforth', 'hope', 'hopeful', 'hopefully', 'hoping', "i'll", "i'mma", 'ima', 'imma', 'imminent', 'impending', "it'll", 'itll', 'looming', 'may', 'might', 'obtainable', 'oncoming', 'onward', 'pray', 'prayed', 'praying', 'promising', 'shall', "shan't", 'shant', "she'll", 'someday', 'sometime', 'soon', 'sooner', 'soonest', "that'll", 'thatll', 'Future', 'will', 'is heading', 'are heading', 'aim', 'could', 'doubt', 'has begun', 'intend', 'intends', 'intending', 'is expected', 'is likely to', 'it is hoped', 'optimism', 'pessimism', 'uncertainty', 'projected', 'certainty', 'as yet', 'not yet', 'may', 'outlook', 'should', 'risk', 'risky', 'risk-taking', 'is planned', 'is planning', 'is scheduled', 'a planned', 'the planned', 'optimistic', 'pes

# Prep + helper functions

### Input and Output Data Location

In [120]:
directory = "/Users/carlyknight/Documents/Data/Annual Report/XML/"
results= "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"
results_paragraphs = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts_paragraphs/"
test_files= "/Users/carlyknight/Dropbox/PROJECTS/ConceptionsofRisk/data/keyword_test_files/"
test_paragraphs = "/Users/carlyknight/Dropbox/PROJECTS/Corporate_Futures_for_Hesu/paragraphs/"

### Help function: spell checker

In [121]:
#function to calculate the percent
def calculate_percent_correct(text):
	true =0
	total = 0.0
	for word in tknzr(text):
		word1 =word[0]
		total +=1
		if d2.check(word1):
			true +=1
	if total !=0:
		percent = true*1.0/total
	else:
		percent =0
	return percent

'''INITIATE SPELLCHECKER'''
d2 = enchant.DictWithPWL("en_US")
tknzr = get_tokenizer("en_US")

### Helper function: clean text

In [122]:
def get_fulltext(root):
	full = root.find('FullText')
	if full is None:
		fulltext = ""
	else:
		fulltext=full.text
	return fulltext

In [123]:
def clean_punction(fulltext):
    #remove various kinds of superfluous punction
    fulltext = re.sub(r'[;:$-*><=|^·\,\\-]','',fulltext)
    fulltext.replace(u'No.', u'No')
    fulltext.replace("\\", "") 
    #replace series of periods with one period
    fulltext = re.sub(r'(\.\s*\.\s*)+', '', fulltext)
    return fulltext

### Regex Compile

In [124]:
pattern_word = r"(?<=\W)(%s)\b" % "|".join(futurelist_word)
regex_word = re.compile(pattern_word)

pattern_stem = r"(?<=\W)(%s)" % "|".join(futurelist_stem)
regex_stem = re.compile(pattern_stem)

pattern_finance = r"(?<=\W)(%s)" % "|".join(financialstatement)
regex_financestatement = re.compile(pattern_finance)

#note: we don't care about case sensitivity for the removals
pattern_remove = r"(?<=\W)(%s)" % "|".join(phrasetoexclude)
regex_remove = re.compile(pattern_remove, re.IGNORECASE)

### Helper fuction: Wordsearch functions

In [125]:
#identify paragraphs
def paragraph_split(fulltext):
    splits= re.split('(\.{3,}[-_\s]*\.{2,}[-_\s]*)|(\_{3,})|(\-{3,})|(\s{3,}[0-9]{1,}\s{3,})|(\s{5,}[A-Z\s]+\s{5,})', fulltext)
    splits=[split for split in splits if split is not None if split.strip() is not '']
    return splits

#combine paragraphs if they are too short
def paragraph_combine(si, length):
    si = iter(si)
    while True:
        try:
            current = next(si)
        except StopIteration:
            return
        try:
            while len(current) < length:
                current += ' ' + next(si)
        except StopIteration:
            return
        try:
            yield current
        except StopIteration:
            return
        

In [126]:
#identify future paragraphs
def paragraph_wordsearch(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #search paragraph for wordlist
        num_matches=len(regex_stem.findall(paragraph)) + len(regex_word.findall(paragraph))
        if num_matches > 0: #if there is a match
            #chain the word and stem iterators:
            chain_iter = chain(regex_stem.finditer(paragraph), regex_word.finditer(paragraph))
            #iterate through chain
            for match in chain_iter:
                #make sure only match is not in not-match list:
                if [m for m in notwordlist if m in paragraph[match.start()-10: match.end()+10]]:
                    continue
                else:
                    #replace with asterisks
                    paragraph =paragraph.replace(match.group(), "***" + match.group())
            #substitute due to loop problems
            paragraph = re.sub(r'\*+', "***", paragraph)
            #if there are asterisks, then append
            if "***" in paragraph:
                selected_paragraphs.append(paragraph)
    return selected_paragraphs


In [127]:
#remove paragraphs contain content we know we don't want
def paragraph_remove(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #search for what to remove
        if len(regex_remove.findall(paragraph))>0: #if there is something to remove
            #print("FOUND PARAGRAPH TO REMOVE")
            continue
        elif "NOTE" in paragraph:
            #print("WORD 'NOTE' FOUND")
            continue
        else:
            selected_paragraphs.append(paragraph)
    return selected_paragraphs


In [128]:
#identify future paragraphs for examples
def paragraph_example_wordsearch(selected_text):
    paragraphs = selected_text.split("####")
    for paragraph in paragraphs: #iterate through paragraph
        for i,item in enumerate(example_list): #for each paragraph, test for each item
            if re.search(r"(?<=\*\*\*)" + item + r"\b", paragraph):
                example_container_lists[i].append(paragraph)

In [129]:
#find financial statement and remove text after
def find_financialstatement(fulltext):
    matches = regex_financestatement.finditer(fulltext)
    text_length = len(fulltext)
    if matches: 
        for m in matches:
            location = m.start()
            finance_location = location/text_length
            #print(m.start(), m.group(), finance_location)
            #if finance location in back fourth of document, remove everything after that location
            if finance_location >= 2/3:
                fulltext = fulltext[:location]
                return(fulltext)
                break
    return(fulltext)

### Helper function: get metadata

In [130]:
def get_metadata(root):
    date= root.find('AlphaPubDate').text
    datenum = root.find('NumericPubDate').text
    year = parser.parse(date).year
    url = root.find('URLDocView').text
    child = root.find('Terms')
    #create empty containers for terms
    company = ''
    naics=''
    code = ''
    classic = ''
    #iterate through terms
    for alltags in child.findall('.//'):
        if alltags.tag == "CompanyName":
            company = alltags.text
        elif alltags.tag == "CompanyNAIC":
            naics = alltags.text
        elif alltags.tag == "ClassCode":
            code = alltags.text
        elif alltags.tag == "ClassExpansion":
            classic = alltags.text
    # put the flexterms into a dictionary
    dict = {}
    for elem in root.iter('FlexTerm'):
        dict[elem.find('FlexTermName').text] = elem.find('FlexTermValue').text
    # create terms from dictionary
    ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns = [dict.get(k) for k in
                                                                            ['AnrClassCode1IdxLit',
                                                                             'AnrCashDispIdxNum',
                                                                             'AnrCashSrchIdxNum',
                                                                             'AnrAssetDispIdxNum',
                                                                             'AnrAssetSrchIdxNum',
                                                                             'AnrRevDispIdxNum',
                                                                             'AnrRevSrchIdxNum',
                                                                             'AnrEarnDispIdxNum',
                                                                             'AnrEarnSrchIdxNum']]
    # if variables above are NONE, recode as empty string
    ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns = [k or "" for k in
                                                                            [ancode, ancashd, ancashs, anasd,
                                                                             anass, anrevd, anrevs, anearnd,
                                                                             anearns]]
    #create list of alternative company names
    altnames =[]
    for elem in root.iter('FlexTerm'):
        if elem.find('FlexTermName').text == "AnrRelCoNameIdxLit":
            altnames.append(elem.find('FlexTermValue').text)
    relnames='; '.join(altnames)
    return date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames
    

# Main loop

### create list for random selection to check

In [131]:
random_selection = []

### create list of zip-files

In [132]:
zippaths=[]
for zipped in os.listdir(directory):
	if not zipped.endswith('.zip'):continue
	zippaths.append(os.path.join(directory, zipped))


### open csv for metadata- paragraph

In [133]:
csvfile = open(results + 'metadata_futureperfect_paragraphs.csv', 'w')
fieldnames = ['Location', 'Filename', "Paragraph", 'RecordTitle', 'Date', 'Date1', 'Year', 'URL', 'CompanyName', 'NAIC', 'ClassCode', 'Classification', 'AnrAssetDispIdxNum', 'numwordstotal', "numwordsparagraph", 'num_paragraphs_prespellcheck', 'num_paragraphs_postspellcheck']
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
writer.writeheader()
            

# TEST

In [134]:
# zip1 = [zippaths[4]]
# for filename in zip1:
#     with ZipFile(filename, "r") as zf:
#         for name in zf.namelist():
#             print("FILE :", name)
#             data= zf.read(name)
#             root = ET.fromstring(data)
#             date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames= get_metadata(root)
#             #get fulltext
#             fulltext= get_fulltext(root)
#             #clean fulltext
#             fulltext=clean_punction(fulltext)
#             #find and remove financial statements later in document
#             fulltext=find_financialstatement(fulltext)
#             #identify paragraphs
#             paragraphs1 = paragraph_split(fulltext)
#             paragraphs2 = list(paragraph_combine(paragraphs1, 200))
#             #wordsearch paragraphs
#             paragraphs3= paragraph_wordsearch(paragraphs2)
#             n_paragraphs_pre= len(paragraphs3)
#             #select only quality paragraphs
#             paragraphs4 = select_good_paragraphs(paragraphs3)
#             #remove bad phrases
#             paragraphs5 = paragraph_remove(paragraphs4)
#             if paragraphs5 is None or len(paragraphs5)==0:
#                 print(str(i), name, "No results returned")
#                 continue
#             #combine paragraphs again for STM analysis (into 500 words)
#             paragraphs6 = paragraph_combine(paragraphs5, 500)
           

# Loop

In [145]:
i=1
for filename in zippaths:
    with ZipFile(filename, "r") as zf:
        for name in zf.namelist():
            data= zf.read(name)
            root = ET.fromstring(data)
            date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames= get_metadata(root)
            #get fulltext
            fulltext= get_fulltext(root)
            #clean fulltext
            fulltext=clean_punction(fulltext)
            #find and remove financial statements later in document
            fulltext=find_financialstatement(fulltext)
            #identify paragraphs
            paragraphs1 = paragraph_split(fulltext)
            paragraphs2 = list(paragraph_combine(paragraphs1, 200))
            #wordsearch paragraphs
            paragraphs3= paragraph_wordsearch(paragraphs2)
            #select only quality paragraphs
            paragraphs4 = select_good_paragraphs(paragraphs3)
            #remove bad phrases
            paragraphs5 = paragraph_remove(paragraphs4)
            #combine paragraphs again for STM analysis (into 500 words)
            paragraphs6 = list(paragraph_combine(paragraphs5, 500))
            n_paragraphs_post= len(paragraphs6)
            if paragraphs6 is None or len(paragraphs6)==0:
                print(str(i), name, "No results returned")
                continue
            #select a random paragraph for Barbara analysis in post 1900 period
            if year >1990:
                selection = random.choice(paragraphs6)
                random_selection.append([name, selection])
            #paragraph_example_wordsearch(selected_text)
            #join text
            selected_text= "####".join(paragraphs6)
            #calculate number of words
            numwordstotal = len(selected_text.split())
            #write out full document
            with open(results + re.sub(".xml", ".txt", name), "w") as text_file:
                _=text_file.write(selected_text)
            #iterate through paragraphs and write file
            for p, paragraph in enumerate(paragraphs6):
                #length of paragraph
                numwordspara = len(paragraph)
                #put results in a dict
                resultsdict= {'Filename': name, "Paragraph": p, 'RecordTitle': id, 'Date': date, "Date1": datenum, 'Year': year, 'URL': url, 'CompanyName': company.encode('utf-8'), 'NAIC': naics, 'ClassCode': code, 'Classification': classic.encode('utf-8'), 'AnrAssetDispIdxNum': anasd, 'numwordstotal': numwordstotal, 'numwordsparagraph': numwordspara, 'num_paragraphs_prespellcheck': n_paragraphs_pre, 'num_paragraphs_postspellcheck': n_paragraphs_post}
                #write to metadata file
                writer.writerow(resultsdict)
                #write out selected text
                filenm = re.sub(".xml", "", name)
                pname = filenm + "_P" + str(p) + ".txt"
                with open(results_paragraphs + pname, "w") as text_file2:
                    _=text_file2.write(paragraph)
            print(str(i), name, "Written")
            i+=1
            
            

1 88201676.xml Written
2 88179860.xml Written
3 88198679.xml Written
4 88190194.xml Written
5 88178029.xml Written
6 88198719.xml Written
7 88177949.xml Written
8 88180180.xml Written
9 88205316.xml Written
10 88180349.xml Written
11 88203156.xml Written
12 88179900.xml Written
13 88179100.xml Written
14 88178669.xml No results returned
14 88175669.xml Written
15 88197156.xml Written
16 88177269.xml No results returned
16 88201519.xml Written
17 88193154.xml Written
18 88200119.xml Written
19 88179620.xml Written
20 88176989.xml No results returned
20 88178229.xml No results returned
20 88191354.xml Written
21 88179420.xml Written
22 88197999.xml Written
23 88176869.xml Written
24 88189354.xml Written
25 88175429.xml Written
26 88204516.xml Written
27 88178909.xml Written
28 88173700.xml Written
29 88180420.xml No results returned
29 88177220.xml Written
30 88176909.xml Written
31 88174700.xml Written
32 88180900.xml Written
33 88175509.xml Written
34 88178069.xml Written
35 88180389.x

277 88195120.xml Written
278 88173051.xml Written
279 88191834.xml Written
280 88172895.xml No results returned
280 88214451.xml Written
281 88213811.xml Written
282 88193074.xml Written
283 88208091.xml Written
284 88183627.xml Written
285 88172699.xml Written
286 88183267.xml No results returned
286 88174055.xml Written
287 88205371.xml Written
288 88181268.xml Written
289 88201451.xml Written
290 88208105.xml Written
291 88205942.xml Written
292 88208185.xml Written
293 88175868.xml No results returned
293 88177308.xml No results returned
293 88173335.xml No results returned
293 88199980.xml Written
294 88181307.xml Written
295 88190634.xml Written
296 88179387.xml No results returned
296 88182788.xml Written
297 88205611.xml Written
298 88191714.xml Written
299 88176588.xml No results returned
299 88191394.xml Written
300 88172977.xml Written
301 88176948.xml Written
302 88207785.xml Written
303 88185707.xml Written
304 88182588.xml Written
305 88173497.xml No results returned
305 

543 88173131.xml Written
544 88173571.xml Written
545 88177371.xml Written
546 88205756.xml Written
547 88209476.xml Written
548 88224356.xml Written
549 88225916.xml Written
550 88216596.xml Written
551 88176912.xml Written
552 88206116.xml Written
553 88223676.xml Written
554 88207156.xml Written
555 88178632.xml Written
556 88207076.xml Written
557 88218116.xml Written
558 88208236.xml Written
559 88223996.xml Written
560 88207756.xml Written
561 88223116.xml Written
562 88220916.xml Written
563 88222116.xml Written
564 88228836.xml Written
565 88181913.xml Written
566 88228803.xml Written
567 88181393.xml No results returned
567 88223756.xml Written
568 88180352.xml Written
569 88180032.xml Written
570 88228916.xml Written
571 88212916.xml Written
572 88223316.xml Written
573 88181193.xml Written
574 88180913.xml Written
575 88206516.xml Written
576 88208476.xml Written
577 88206076.xml Written
578 88223569.xml Written
579 88176872.xml No results returned
579 88180832.xml Written
5

819 88213393.xml Written
820 88204232.xml Written
821 88193723.xml Written
822 88200392.xml Written
823 88184816.xml Written
824 88196043.xml No results returned
824 88205075.xml Written
825 88204115.xml Written
826 88197392.xml Written
827 88195643.xml Written
828 88203192.xml Written
829 88189518.xml Written
830 88189798.xml Written
831 88199312.xml Written
832 88187005.xml Written
833 88231164.xml Written
834 88204194.xml Written
835 88180457.xml Written
836 88177777.xml No results returned
836 88176897.xml Written
837 88180174.xml Written
838 88202438.xml Written
839 88190758.xml Written
840 88214233.xml Written
841 88186678.xml Written
842 88192963.xml Written
843 88197123.xml Written
844 88198598.xml Written
845 88192118.xml Written
846 88198672.xml Written
847 88193523.xml Written
848 88188099.xml Written
849 88187958.xml Written
850 88220553.xml Written
851 88183979.xml No results returned
851 88206152.xml Written
852 88205352.xml Written
853 88213433.xml Written
854 88221004.x

1112 88174744.xml Written
1113 88175865.xml Written
1114 88178304.xml No results returned
1114 88178384.xml Written
1115 88175225.xml Written
1116 88177745.xml No results returned
1116 88174584.xml Written
1117 88173780.xml Written
1118 88174820.xml No results returned
1118 88176220.xml Written
1119 88176345.xml Written
1120 88174300.xml Written
1121 88176465.xml No results returned
1121 88175145.xml Written
1122 88175466.xml No results returned
1122 88174824.xml No results returned
1122 88176705.xml No results returned
1122 88176062.xml No results returned
1122 88175224.xml Written
1123 88173620.xml Written
1124 88176624.xml Written
1125 88175265.xml Written
1126 88175186.xml Written
1127 88176184.xml Written
1128 88177585.xml No results returned
1128 88174985.xml No results returned
1128 88199055.xml Written
1129 88201375.xml Written
1130 88199530.xml Written
1131 88182081.xml No results returned
1131 88184841.xml Written
1132 88183187.xml Written
1133 88179948.xml No results returne

...

38550 88204493.xml Written
38551 88207693.xml Written
38552 88197141.xml Written
38553 88206508.xml Written
38554 88203372.xml Written
38555 88196141.xml Written
38556 88184022.xml Written
38557 88189863.xml Written
38558 88187645.xml Written
38559 88188707.xml Written
38560 88194504.xml Written
38561 88222841.xml Written
38562 88199492.xml Written
38563 88185183.xml Written
38564 88194704.xml Written
38565 88185103.xml Written
38566 88224041.xml Written
38567 88191023.xml Written
38568 88194307.xml Written
38569 88194787.xml Written
38570 88195467.xml Written
38571 88201332.xml Written
38572 88185982.xml Written
38573 88207093.xml Written
38574 88188747.xml No results returned
38574 88225141.xml Written
38575 88208293.xml Written
38576 88202212.xml Written
38577 88189427.xml Written
38578 88221493.xml Written
38579 88187747.xml Written
38580 88205628.xml Written
38581 88187383.xml Written
38582 88229321.xml Written
38583 88196661.xml Written
38584 88203013.xml Written
38585 88191307.x

38827 88186762.xml Written
38828 88213250.xml Written
38829 88180867.xml No results returned
38829 88218487.xml Written
38830 88187069.xml Written
38831 88208170.xml Written
38832 88207803.xml Written
38833 88185307.xml Written
38834 88206410.xml Written
38835 88203490.xml Written
38836 88189405.xml Written
38837 88195322.xml Written
38838 88213003.xml Written
38839 88199250.xml Written
38840 88188709.xml Written
38841 88205723.xml Written
38842 88191405.xml Written
38843 88207763.xml Written
38844 88176227.xml Written
38845 88192452.xml Written
38846 88204960.xml Written
38847 88217087.xml Written
38848 88198883.xml Written
38849 88214810.xml Written
38850 88191682.xml Written
38851 88199775.xml Written
38852 88209167.xml Written
38853 88208043.xml Written
38854 88195162.xml Written
38855 88175125.xml Written
38856 88224219.xml Written
38857 88187109.xml Written
38858 88180827.xml Written
38859 88200370.xml Written
38860 88188962.xml Written
38861 88174125.xml Written
38862 88176366.x

39119 88220595.xml Written
39120 88218541.xml Written
39121 88207950.xml Written
39122 88226875.xml Written
39123 88186898.xml No results returned
39123 88180615.xml Written
39124 88182335.xml Written
39125 88206790.xml Written
39126 88229552.xml Written
39127 88182148.xml Written
39128 88185145.xml Written
39129 88215294.xml Written
39130 88187386.xml No results returned
39130 88185770.xml Written
39131 88189650.xml Written
39132 88190530.xml Written
39133 88207590.xml Written
39134 88211472.xml Written
39135 88191371.xml Written
39136 88202024.xml Written
39137 88181450.xml Written
39138 88181628.xml Written
39139 88190570.xml Written
39140 88220899.xml Written
39141 88181170.xml Written
39142 88228715.xml Written
39143 88188611.xml Written
39144 88202904.xml Written
39145 88229744.xml Written
39146 88203468.xml Written
39147 88188546.xml Written
39148 88211872.xml Written
39149 88192211.xml Written
39150 88183090.xml Written
39151 88191531.xml Written
39152 88180135.xml Written
3915

39381 88173361.xml Written
39382 88179743.xml No results returned
39382 88186001.xml Written
39383 88220106.xml Written
39384 88195408.xml Written
39385 88199769.xml Written
39386 88186928.xml Written
39387 88180663.xml No results returned
39387 88178301.xml No results returned
39387 88201371.xml Written
39388 88191774.xml Written
39389 88225641.xml Written
39390 88186965.xml Written
39391 88186485.xml Written
39392 88184321.xml Written
39393 88201936.xml Written
39394 88185688.xml Written
39395 88202609.xml Written
39396 88204656.xml Written
39397 88181901.xml Written
39398 88205296.xml Written
39399 88206626.xml Written
39400 88189945.xml Written
39401 88178661.xml Written
39402 88200726.xml Written
39403 88177541.xml No results returned
39403 88194174.xml Written
39404 88192041.xml Written
39405 88214266.xml Written
39406 88175743.xml Written
39407 88202699.xml Written
39408 88180641.xml Written
39409 88184205.xml Written
39410 88212906.xml Written
39411 88190774.xml Written
39412 8

39665 88228860.xml Written
39666 88230869.xml Written
39667 88229143.xml Written
39668 88228949.xml Written
39669 88224143.xml Written
39670 88214140.xml Written
39671 88229669.xml Written
39672 88223463.xml Written
39673 88226143.xml Written
39674 88228749.xml Written
39675 88225143.xml Written
39676 88231306.xml Written
39677 88228500.xml Written
39678 88229786.xml Written
39679 88228709.xml Written
39680 88209380.xml Written
39681 88230226.xml Written
39682 88224650.xml Written
39683 88199562.xml Written
39684 88209720.xml Written
39685 88207482.xml No results returned
39685 88228573.xml Written
39686 88229892.xml Written
39687 88229165.xml Written
39688 88213890.xml Written
39689 88221180.xml Written
39690 88208290.xml Written
39691 88206420.xml Written
39692 88209722.xml Written
39693 88218762.xml Written
39694 88206842.xml Written
39695 88209280.xml Written
39696 88202842.xml Written
39697 88221850.xml Written
39698 88212460.xml Written
39699 88201242.xml Written
39700 88206285.x

## Create text file with examples

In [192]:
#for i,item in enumerate(example_list):
#    with open(test_files +  item.replace(" ", "") + '2.txt', 'w') as f:
#        f.write('%s\n\n' % example_container_lists[i])

## Create selection of 1000 random paragrahs

In [146]:
#how many were selected?
len(random_selection)

6220

In [147]:
#select 1000 random paragraphs 
random_sample = random.choices(random_selection, k=1000)
len(random_sample)

1000

In [148]:
#put these in a csv
with open(test_paragraphs + "random_sample_27JUNE.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(random_sample)